In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt

from turtle_quant_1.data_processing.processor import DataProcessor
from turtle_quant_1.strategies.helpers.helpers import convert_to_weekly_data
from turtle_quant_1.strategies.helpers.support_resistance import (
    StnryFibonacciRetrace,
    StnryGaussianKde,
    StnryLocalExtrema,
    StnryPivotPoint,
)

## Data pre-processing

In [ ]:
dates = {"start": datetime(2024, 1, 1), "end": datetime(2025, 7, 31)}

data_processor = DataProcessor()
data = data_processor.load_data(
    symbol="MSFT",
    start_date=dates["start"],
    end_date=dates["end"],
    impute_data=True,
)

In [ ]:
# This step is not necessary, 
# but just so that it is easier to see what data the strategies are processing
# if they are designed to process daily / weekly data
agg_data = convert_to_weekly_data(data)

## Select strategies to test

In [ ]:
strategies = [
    StnryFibonacciRetrace(),
    StnryGaussianKde(),
    StnryLocalExtrema(),
    StnryPivotPoint(),
]

## Visualization

In [ ]:
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots

for strategy in strategies:
    # Adjust this to configure what price counts as being within a support-resistance zone
    sup_res_zone_threshold = strategy.sup_res_zone_threshold
    
    levels_df = strategy.generate_historical_levels(data=data, symbol="MSFT")
    levels_df = strategy.pivoted_levels(levels_df)
    
    fig = make_subplots(
        rows=1,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        row_heights=[1.0],
        subplot_titles=("Close Price",),
    )

    # Price plot with original data
    fig.add_trace(
        go.Scatter(
            x=data["datetime"],
            y=data["Close"],
            mode="lines",
            name="Close Price",
            line=dict(color="blue"),
        ),
        row=1,
        col=1,
    )

    # Price plot with aggregated data
    fig.add_trace(
        go.Scatter(
            x=agg_data["datetime"],
            y=agg_data["Close"],
            mode="lines",
            name="Close Price (Aggregated)",
            line=dict(color="green"),
        ),
        row=1,
        col=1,
    )

    # Plot support/resistance levels
    for _, row in levels_df.iterrows():
        for level in row["level_values"]:
            if level == 0.0:
                continue
            
            datetime_beg = row["datetime_beg"]
            datetime_end = row["datetime_end"]
            lower = level * (1 - sup_res_zone_threshold)
            upper = level * (1 + sup_res_zone_threshold)
    
            # Add the filled zone
            fig.add_trace(
                go.Scatter(
                    x=[datetime_beg, datetime_end, datetime_end, datetime_beg],
                    y=[lower, lower, upper, upper],
                    fill='toself',
                    fillcolor='rgba(255,165,0,0.1)',  # semi-transparent orange
                    line=dict(color='rgba(255,165,0,0.0)'),  # no border
                    hoverinfo='skip',
                    showlegend=False,
                )
            )
    
            # Add the central line
            fig.add_trace(
                go.Scatter(
                    x=[datetime_beg, datetime_end],
                    y=[level, level],
                    mode="lines",
                    line=dict(color="orange", width=1),
                    name=f"Level {level:.2f}",
                    showlegend=False,
                )
            )

    # Layout
    fig.update_layout(
        height=600,
        width=1000,
        title_text=f"Support & Resistance - {type(strategy).__name__} - MSFT",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    fig.update_yaxes(title_text="Price", row=1, col=1)

    display(fig)